<!-- @format -->

# Importing Libraries


In [1]:
import numpy as np
import random
import math
import hashlib
import secrets
from sympy import nextprime, isprime, sqrt_mod, isprime
import subprocess

<!-- @format -->

# Functions

## 1 - Greatest Common Divisor

The greatest common divisor of $a$ and $b$ is calculated via Extended Euclidean Algorithm.


In [2]:
def gcd(a, b):
    s0, s1 = 1, 0
    t0, t1 = 0, 1
    while b:
        q = a // b
        s1, s0 = s0 - q * s1, s1
        t1, t0 = t0 - q * t1, t1
        a, b = b, a % b
    return a, s0, t0

<!-- @format -->

## 2- Modulo Inverse

The modulo inverse of $a$ modulo $m$ is denoted as $a^{-1}$ and defined as
\begin{equation*}
a \cdot a^{-1} \equiv 1 \mod{m} \;\Longleftrightarrow\; \text{gcd}(a, m) = 1.
\end{equation*}


In [3]:
def modulo_inverse(a, m):
    (gcd_, s, t) = gcd(a, m)
    return s % m if gcd_ == 1 else 0

<!-- @format -->

## 3 - Point Addition

The addition of two points $P = (x_1, y_1)$ and $Q = (x_2, y_2)$ on an elliptic curve $E$ defined over the finite field $\mathbb{F}_p$ is defined as

-   if either $P = \mathcal{O}$ or $Q = \mathcal{O}$, then $P + Q = P$ or $P + Q = Q$, respectively.
-   if $P = -Q$, then $P + Q = \mathcal{O}$.
-   if $P = Q$, then $P + Q = (x_3, y_3)$, where
    \begin{equation*}
    x_3 = \left(\frac{3x_1^2 + a}{2y_1}\right)^2 - 2x_1 \quad \text{and} \quad y_3 = \left(\frac{3x_1^2 + a}{2y_1}\right)(x_1 - x_3) - y_1.
    \end{equation*}
-   if $P \neq  \pm Q$, then $P + Q = (x_3, y_3)$, where
    \begin{equation*}
    x_3 = \left(\frac{y_2 - y_1}{x_2 - x_1}\right)^2 - x_1 - x_2 \quad \text{and} \quad y_3 = \left(\frac{y_2 - y_1}{x_2 - x_1}\right)(x_1 - x_3) - y_1.
    \end{equation*}


In [4]:
def point_addition(x1, y1, x2, y2, a, p):
    if x1 == 0 and y1 == 0:
        return x2, y2
    elif x2 == 0 and y2 == 0:
        return x1, y1
    elif x1 == x2 and y1 == -y2 % p:
        return 0, 0
    elif x1 == x2 and y1 == y2:
        s = (3 * x1**2 + a) * modulo_inverse(2 * y1, p)
        x3 = s**2 - 2 * x1
        y3 = s * (x1 - x3) - y1
    else:
        s = ((y2 - y1) * modulo_inverse(x2 - x1, p)) % p
        x3 = s**2 - x1 - x2
        y3 = s * (x1 - x3) - y1

    return x3 % p, y3 % p

<!-- @format -->

## 4 - Scalar Multiplication

The scalar multiplication of a point $P = (x, y)$ on an elliptic curve $E$ defined over the finite field $\mathbb{F}_p$ by an integer $k$ is defined as
\begin{equation*}
kP = \underbrace{P + P + \ldots + P}\_{k \ \text{times}} \ \ \text{for} \ k > 0, \ \text{and} \ 0P = \mathcal{O}.
\end{equation*}


In [5]:
def scalar_multiplication(x, y, k, a, p):
    if k == 0:
        return 0, 0
    if k == 1:
        return x, y

    qx, qy = 0, 0
    k = format(k, "b")

    for bit in k:
        qx, qy = point_addition(qx, qy, qx, qy, a, p)
        if int(bit) & 1:
            qx, qy = point_addition(qx, qy, x, y, a, p)

    return qx, qy

<!-- @format -->

## 5 - Is On Curve

A point $P = (x, y)$ is on an elliptic curve $E$ defined over the finite field $\mathbb{F}_p$ if and only if
\begin{equation*}
y^2 \equiv x^3 + ax + b \mod{p}.
\end{equation*}


In [6]:
def is_on_curve(x, y, a, b, p):
    return (y**2 - x**3 - a * x - b) % p == 0

<!-- @format -->

## 6 - Is Curve Valid

An elliptic curve $E$ defined over the finite field $\mathbb{F}_p$ is valid if and only if
\begin{equation*}
4a^3 + 27b^2 \not\equiv 0 \mod{p}.
\end{equation*}


In [7]:
def is_curve_valid(a, b, p):
    return (4 * a**3 + 27 * b**2) % p != 0

<!-- @format -->

## 7 - Order Of A Point

The order of a point $P = (x, y)$ on an elliptic curve $E$ defined over the finite field $\mathbb{F}_p$ is defined as the smallest positive integer $k$ such that
\begin{equation*}
kP = \mathcal{O}.
\end{equation*}


In [8]:
def order_of_point(x, y, a, b, p):
    try:
        result = subprocess.run(
            [
                "sage",
                "utils.sage",
                str(p),
                str(a),
                str(b),
                "-x",
                str(x),
                "-y",
                str(y),
                "-o",
            ],
            stdout=subprocess.PIPE,
        )
        return int(result.stdout.decode("utf-8"))
    except:
        return 0

<!-- @format -->

## 8 - Number Of Points

The number of points function returns the number of points on an elliptic curve $E$ defined over the finite field $\mathbb{F}_p$.


In [9]:
def number_of_points(a, b, p):
    try:
        result = subprocess.run(
            ["sage", "utils.sage", str(p), str(a), str(b), "-c"], stdout=subprocess.PIPE
        )
        return int(result.stdout.decode("utf-8"))
    except:
        return 0

<!-- @format -->

## 9 - Cofactor Of The Curve

The cofactor of an elliptic curve $E$ defined over the finite field $\mathbb{F}_p$ is defined as
\begin{equation*}
h = \frac{\#E(\mathbb{F}\_p)}{n},
\end{equation*}
where $n$ is the order of the base point $P$.


In [10]:
def cofactor_of_curve(n, N):
    cofactor = N // n
    return cofactor

<!-- @format -->

## 10 - Key Generation

The key generation function takes elliptic curve public domain parameters and generates a private key $d$ and a public key $Q$.


In [11]:
def key_generation(a, p, P, n):
    d = random.randint(1, n - 1)
    qx, qy = scalar_multiplication(P[0], P[1], d, a, p)
    return (qx, qy), d

<!-- @format -->

## 11 - ECDSA Signature Generation

The ECDSA signature generation function takes a private key $d$, a message $m$, and elliptic curve public domain parameters and generates a signature $(r, s)$.


In [12]:
def ecdsa_sign(a, p, P, n, d, m):
    r, s = 0, 0
    while r == 0 or s == 0 or modulo_inverse(s, n) == 0:
        k = random.randint(1, n - 1)
        qx, _ = scalar_multiplication(P[0], P[1], k, a, p)
        r = qx % n
        if r == 0:
            continue
        e = hashlib.sha256(m.encode()).hexdigest()
        e = int(e, 16)
        s = (modulo_inverse(k, n) * (e + d * r)) % n
    return (r, s)

<!-- @format -->

## 12 - ECDSA Signature Verification

The ECDSA signature verification function takes a public key $Q$, a message $m$, and a signature $(r, s)$ and outputs whether the signature is valid or not.


In [13]:
def ecdsa_verify(a, p, P, n, Q, m, r, s):
    if r < 1 or r > n - 1:
        return False
    if s < 1 or s > n - 1:
        return False
    e = hashlib.sha256(m.encode()).hexdigest()
    e = int(e, 16)
    w = modulo_inverse(s, n)
    u1 = (e * w) % n
    u2 = (r * w) % n
    qx, qy = point_addition(
        *scalar_multiplication(P[0], P[1], u1, a, p),
        *scalar_multiplication(Q[0], Q[1], u2, a, p),
        a,
        p,
    )
    if qx == 0 and qy == 0:
        return False
    return r == qx % n

<!-- @format -->

## 13 - Even Hex

The even hex function takes a hexadecimal string and outputs the same string as even length by prepending a zero if the length of the string is odd.


In [14]:
def even_hex(n):
    prefix = ""

    if "0x" in n:
        n = n[2:]
        prefix = "0x"

    h = n

    if len(h) % 2 == 1:
        h = prefix + "0" + h

    return h

<!-- @format -->

## 14 - Find Integer

The find integer function takes integers $l$ and $n$ and finds a $(l - n)$-bit long integer $x$ using SHA-1.


In [15]:
def find_integer(s, l, n):
    v = math.floor((l - 1) / 256)
    w = l - 256 * v - n
    h = hashlib.sha256(bytearray.fromhex(s)).hexdigest()
    h = np.base_repr(int(h, 16), base=2)
    h0 = h[-w:]
    z = int(s, 16)
    h_array = [h0]

    for i in range(1, v + 1):
        zi = (z + i) % 2**256
        si = bytearray.fromhex(even_hex(np.base_repr(zi, base=16)))
        hi = hashlib.sha256(si).hexdigest()
        hi = np.base_repr(int(hi, 16), base=2)
        h_array.append(hi)

    return int("".join(h_array), 2)

<!-- @format -->

## 15 - Update Seed

The update seed function takes a seed $s$ and updates it to $s'$ using modular arithmetic.


In [16]:
def update_seed(s):
    z = int(s, 16)
    return even_hex(np.base_repr((z + 1) % 2**256, base=16))

<!-- @format -->

## 16 - Legendre Symbol

The Legendre symbol of $a$ modulo $p$ is defined as
\begin{equation*}
\left(\frac{a}{p}\right) = a^{\frac{p - 1}{2}} \mod{p}.
\end{equation*}


In [17]:
def legendre_symbol(a, p):
    return pow(a, (p - 1) // 2, p)

<!-- @format -->

## 17 - Is Fourth Power Residue

An integer $a$ is a fourth power residue modulo $p$ if and only if
\begin{equation*}
\left(\frac{a}{p}\right) = 1 \quad \text{and} \quad a^{\frac{p - 1}{4}} \equiv 1 \mod{p}.
\end{equation*}


In [29]:
def is_fourth_power_residue(A, p):
    a = (-3 * modulo_inverse(A, p)) % p
    f = p - 1
    k = f // (4 if f % 4 == 0 else 2)
    return pow(a, k, p) == 1

<!-- @format -->

## 17 - Generate Prime Number

The generate prime number function generates a random prime number $p$ of a given bit length.


In [19]:
def generate_prime(l, seed=None):
    s = seed if seed else secrets.token_hex(20)

    while True:
        c = find_integer(s, l, 0)

        if isprime(c) and c % 4 == 3:
            p = c
        else:
            i = 1
            while True:
                p = nextprime(c, i)
                if p % 4 == 3:
                    break
                i += 1

        if 2 ** (l - 1) <= p <= 2**l - 1:
            return p

        s = update_seed(s)

<!-- @format -->

## 18 - Generate Elliptic Curve

The generate elliptic curve function takes a prime number $p$ and generates a random elliptic curve $E$ defined over the finite field $\mathbb{F}_p$ and a base point $P$ on the curve.


In [20]:
def generate_curve(p, seed=None):
    s = seed if seed else secrets.token_hex(20)

    while True:
        a = find_integer(s, p.bit_length(), 1)

        if not is_fourth_power_residue(a, p):
            s = update_seed(s)
            continue

        s = update_seed(s)

        while True:
            b = find_integer(s, p.bit_length(), 1)
            if pow(b, (p - 1) // 2, p) == 1:
                s = update_seed(s)
                continue
            break

        if not is_curve_valid(a, b, p):
            s = update_seed(s)
            continue

        s = update_seed(s)
        k = find_integer(s, p.bit_length(), 1)

        for i in range(p):
            x = i
            rhs = pow(x, 3) + a * x + b
            if legendre_symbol(rhs, p) == 1:
                y_values = sqrt_mod(rhs, p, all_roots=True)
                break

        points = [(x, y) for y in y_values]
        P = scalar_multiplication(*(random.choice(points)), k, a, p)
        return a, b, P

<!-- @format -->

# Main Program


In [31]:
p = generate_prime(160)
a, b, P = generate_curve(p)
n = order_of_point(*P, a, b, p)
N = number_of_points(a, b, p)

print("Elliptic Curve Public Domain Parameters")
print("{:-<50}".format(""))
print(
    "Curve: y^2 = x^3 + {}x + {} mod {} ({})".format(
        a, b, p, "Valid" if is_curve_valid(a, b, p) else "Invalid"
    )
)
print("Base point: {}".format(P))
print("Order of base point: {}".format(n))
print("Number of points on curve: {}".format(N))

print("\nKey generation")
print("{:-<50}".format(""))
Q, d = key_generation(a, p, P, n)
print("Private key: {}".format(d))
print("Public key: {}".format(Q))

print("\nECDSA")
print("{:-<50}".format(""))
message = "Hello"
print("Message: {}".format(message))
r, s = ecdsa_sign(a, p, P, n, d, message)
print("Signature: ({}, {})".format(r, s))
print(
    "Signature verification: {}".format(
        "Accepted" if ecdsa_verify(a, p, P, n, Q, message, r, s) else "Rejected"
    )
)

Elliptic Curve Public Domain Parameters
--------------------------------------------------
Curve: y^2 = x^3 + 369418060276964572701710564349681860536433030190x + 217958547323320411081979073871532299660744119247 mod 1124102222762367294231581020193451298170272483263 (Valid)
Base point: (45449531919476556096386997901311655575098027247, 755322846667935462826379310779939648056560323697)
Order of base point: 140512777845295911778947693368915151960028973117
Number of points on curve: 1124102222762367294231581546951321215680231784936

Key generation
--------------------------------------------------
Private key: 68822667255812937596407155387785583742409445927
Public key: (19079279623995735153317667270818615024821618128, 1087977239646571208878209887960740472439859102874)

ECDSA
--------------------------------------------------
Message: Hello
Signature: (97011209971298138876057541259128091530986453933, 55924813801046326936924738230429090439064901560)
Signature verification: Accepted
